<a href="https://colab.research.google.com/github/Vinit-source/piper1-gpl/blob/copilot/create-finetuning-notebook/notebooks/piper_finetuning_indian_english_from_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Piper TTS Fine-Tuning: Indian English Voice (Google Drive Edition)

This notebook provides a complete pipeline for fine-tuning the Piper TTS model
using a dataset stored in Google Drive.

**Compatible with:** Google Colab

## Overview
1. Colab Anti-Disconnect
2. Check GPU Type
3. Mount Google Drive
4. Install Software
5. Download Dataset from Drive (audio + transcript)
6. Training
7. Save Model and Config to Drive

---

**Repository:** https://github.com/Vinit-source/piper1-gpl

# 🔧 **1. First Steps** 🔧

In [ ]:
#@markdown ## **Google Colab Anti-Disconnect.** 🔌
#@markdown ---
#@markdown #### Avoid automatic disconnection. Still, it will disconnect after **6 to 12 hours**.

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@markdown ## **Check GPU Type.** 👁️
#@markdown ---
#@markdown #### A higher capable GPU can lead to faster training speeds. By default, you will have a **Tesla T4**.
!nvidia-smi

In [ ]:
#@markdown # **Mount Google Drive.** 📂
#@markdown ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 📦 **2. Install Software** 📦

In [ ]:
#@markdown # **Install Piper and Dependencies.** 📦
#@markdown ---
#@markdown This cell installs Piper TTS and all necessary dependencies for training.

import os

# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq espeak-ng build-essential cmake ninja-build

# Clone the piper1-gpl repository with updated dependencies
PIPER_REPO_URL = "https://github.com/Vinit-source/piper1-gpl.git"
PIPER_BRANCH = "copilot/fine-tune-text-to-speech-model"
PIPER_DIR = "/content/piper1-gpl"

if not os.path.exists(PIPER_DIR):
    !git clone -b {PIPER_BRANCH} {PIPER_REPO_URL} {PIPER_DIR}
else:
    print(f"Repository already exists at {PIPER_DIR}")

# Install piper with training dependencies
%cd {PIPER_DIR}
!pip install -e .[train] -q

# Build monotonic alignment module (required for VITS)
!bash build_monotonic_align.sh

print("\n✅ Piper TTS installed successfully!")

# 📥 **3. Download Dataset from Drive** 📥

In [ ]:
#@markdown # **Configure Dataset Path.** 📁
#@markdown ---
#@markdown ### Set the path to your dataset in Google Drive.
#@markdown
#@markdown **Expected folder structure:**
#@markdown ```
#@markdown your_dataset_folder/
#@markdown ├── wavs/           # Audio files (WAV format, 22050Hz recommended)
#@markdown │   ├── 1.wav
#@markdown │   ├── 2.wav
#@markdown │   └── ...
#@markdown └── metadata.csv    # Transcript file (format: wavs/filename.wav|text)
#@markdown ```
#@markdown
#@markdown **Transcript format (single speaker):**
#@markdown ```
#@markdown wavs/1.wav|This is the text for audio 1.
#@markdown wavs/2.wav|This is the text for audio 2.
#@markdown ```
#@markdown ---

#@markdown ### Path to your dataset folder in Google Drive:
gdrive_dataset_path = "/content/drive/MyDrive/Piper-Training/dataset" #@param {type:"string"}

#@markdown ### Model name (used for output files):
model_name = "en_IN-spicor-medium" #@param {type:"string"}

#@markdown ### Output path in Google Drive (for saving checkpoints and models):
gdrive_output_path = "/content/drive/MyDrive/Piper-Training/output" #@param {type:"string"}

#@markdown ---

import os
from pathlib import Path

# Validate paths
gdrive_dataset_path = gdrive_dataset_path.strip()
gdrive_output_path = gdrive_output_path.strip()

# Create local working directories
LOCAL_DATASET_DIR = "/content/dataset"
LOCAL_WAVS_DIR = f"{LOCAL_DATASET_DIR}/wavs"
LOCAL_CACHE_DIR = "/content/audio_cache"
LOCAL_OUTPUT_DIR = f"/content/output/{model_name}"

os.makedirs(LOCAL_DATASET_DIR, exist_ok=True)
os.makedirs(LOCAL_WAVS_DIR, exist_ok=True)
os.makedirs(LOCAL_CACHE_DIR, exist_ok=True)
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

# Create output directory in Google Drive
os.makedirs(gdrive_output_path, exist_ok=True)

print(f"Dataset path: {gdrive_dataset_path}")
print(f"Output path: {gdrive_output_path}")
print(f"Local dataset dir: {LOCAL_DATASET_DIR}")
print(f"Local output dir: {LOCAL_OUTPUT_DIR}")

In [ ]:
#@markdown # **Extract Dataset.** 📥
#@markdown ---
#@markdown This cell copies your dataset from Google Drive to the local environment.
#@markdown
#@markdown **Important:** Audio files must be in WAV format (22050Hz, 16-bit, mono recommended).
#@markdown ---

import os
import wave
import zipfile
import datetime
import shutil

def get_dataset_duration(wav_path):
    """Calculate total duration of all WAV files in a directory."""
    totalduration = 0
    totalcount = 0
    for file_name in os.listdir(wav_path):
        if not file_name.endswith(".wav"):
            continue
        full_path = os.path.join(wav_path, file_name)
        try:
            with wave.open(full_path, "rb") as wave_file:
                frames = wave_file.getnframes()
                rate = wave_file.getframerate()
                duration = frames / float(rate)
                totalduration += duration
                totalcount += 1
        except Exception as e:
            print(f"Skipping bad file {file_name}: {e}")
            continue
    duration_str = str(datetime.timedelta(seconds=round(totalduration, 0)))
    return totalcount, duration_str

def cleanup_ghost_files(dataset_path):
    """Delete macOS ghost files (._*) from the dataset."""
    count = 0
    for filename in os.listdir(dataset_path):
        if filename.startswith("._"):
            file_path = os.path.join(dataset_path, filename)
            os.remove(file_path)
            count += 1
    if count > 0:
        print(f"Cleaned up {count} macOS artifact files.")

# Check if dataset exists in Google Drive
if not os.path.exists(gdrive_dataset_path):
    raise Exception(f"Dataset folder not found: {gdrive_dataset_path}")

# Check for wavs folder or zip file
gdrive_wavs_path = os.path.join(gdrive_dataset_path, "wavs")
gdrive_wavs_zip = os.path.join(gdrive_dataset_path, "wavs.zip")

if os.path.exists(gdrive_wavs_zip):
    print("Found wavs.zip, extracting...")
    with zipfile.ZipFile(gdrive_wavs_zip, 'r') as zip_ref:
        zip_ref.extractall(LOCAL_DATASET_DIR)
    # Handle nested wavs folder if present
    if os.path.exists(f"{LOCAL_DATASET_DIR}/wavs/wavs"):
        for f in os.listdir(f"{LOCAL_DATASET_DIR}/wavs/wavs"):
            shutil.move(f"{LOCAL_DATASET_DIR}/wavs/wavs/{f}", f"{LOCAL_WAVS_DIR}/{f}")
elif os.path.exists(gdrive_wavs_path):
    print("Found wavs folder, copying...")
    !cp -r "{gdrive_wavs_path}"/* "{LOCAL_WAVS_DIR}/"
else:
    raise Exception(f"No 'wavs' folder or 'wavs.zip' found in {gdrive_dataset_path}")

# Clean up macOS ghost files
cleanup_ghost_files(LOCAL_WAVS_DIR)

# Copy transcript file
transcript_files = ["metadata.csv", "transcripts.txt", "transcript.txt", "metadata.txt"]
transcript_found = False

for tf in transcript_files:
    gdrive_transcript = os.path.join(gdrive_dataset_path, tf)
    if os.path.exists(gdrive_transcript):
        print(f"Found transcript file: {tf}")
        shutil.copy(gdrive_transcript, f"{LOCAL_DATASET_DIR}/metadata.csv")
        transcript_found = True
        break

if not transcript_found:
    raise Exception(f"No transcript file found in {gdrive_dataset_path}. Expected one of: {transcript_files}")

# Get dataset statistics
audio_count, dataset_dur = get_dataset_duration(LOCAL_WAVS_DIR)
print(f"\n✅ Dataset loaded: {audio_count} audio files, total duration: {dataset_dur}")

# 🤖 **4. Training** 🤖

In [ ]:
#@markdown # **Training Configuration.** ⚙️
#@markdown ---

#@markdown ### Language of the dataset:
language = "en" #@param ["ar", "ca", "cs", "da", "de", "el", "en", "en-us", "es", "es-419", "fi", "fr", "hu", "is", "it", "ka", "kk", "lb", "ne", "nl", "nb", "pl", "pt-br", "pt-pt", "ro", "ru", "sr", "sv", "sw", "tr", "uk", "vi", "zh"]

#@markdown ### Sample rate (22050 recommended for medium/high quality):
sample_rate = 22050 #@param {type:"integer"}

#@markdown ### Is this a single speaker dataset?
single_speaker = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Training Parameters:

#@markdown ### Batch size (reduce if out of memory):
batch_size = 16 #@param {type:"integer"}

#@markdown ### Maximum epochs:
max_epochs = 1000 #@param {type:"integer"}

#@markdown ### Validation split (0.0 to disable validation):
validation_split = 0.05 #@param {type:"number"}

#@markdown ### Number of test examples for audio generation:
num_test_examples = 3 #@param {type:"integer"}

#@markdown ---
#@markdown ### Fine-tuning Options:

#@markdown ### Download and use a pretrained checkpoint for fine-tuning?
use_pretrained = True #@param {type:"boolean"}

#@markdown ### Resume from existing checkpoint in output folder?
resume_training = False #@param {type:"boolean"}

#@markdown ---

# Calculate num_speakers
num_speakers = 1 if single_speaker else 0  # 0 will be calculated from metadata

print(f"Language: {language}")
print(f"Sample rate: {sample_rate}")
print(f"Batch size: {batch_size}")
print(f"Max epochs: {max_epochs}")
print(f"Validation split: {validation_split}")
print(f"Use pretrained: {use_pretrained}")
print(f"Resume training: {resume_training}")

In [ ]:
#@markdown # **Download Pretrained Checkpoint.** 📥
#@markdown ---
#@markdown Downloads a pretrained checkpoint from Hugging Face for fine-tuning.

import os

PRETRAINED_CKPT_PATH = "/content/pretrained.ckpt"

if use_pretrained and not resume_training:
    if not os.path.exists(PRETRAINED_CKPT_PATH):
        print("Downloading pretrained checkpoint from Hugging Face...")

        # Install huggingface_hub if needed
        !pip install -q huggingface_hub

        from huggingface_hub import hf_hub_download

        # Download the en_US hfc_female medium checkpoint
        downloaded_path = hf_hub_download(
            repo_id="rhasspy/piper-checkpoints",
            filename="en/en_US/hfc_female/medium/epoch=2868-step=1575188.ckpt",
            repo_type="dataset",
            local_dir="/content/checkpoints",
        )

        # Also download the config for reference
        config_path = hf_hub_download(
            repo_id="rhasspy/piper-checkpoints",
            filename="en/en_US/hfc_female/medium/config.json",
            repo_type="dataset",
            local_dir="/content/checkpoints",
        )

        # Copy to expected location
        !cp "{downloaded_path}" "{PRETRAINED_CKPT_PATH}"

        print(f"\n✅ Pretrained checkpoint downloaded to: {PRETRAINED_CKPT_PATH}")
    else:
        print(f"Pretrained checkpoint already exists: {PRETRAINED_CKPT_PATH}")
elif resume_training:
    print("Resume training mode - will look for existing checkpoint in output folder.")
else:
    print("Training from scratch (no pretrained checkpoint).")

In [ ]:
#@markdown # **Launch TensorBoard.** 📊
#@markdown ---
#@markdown TensorBoard allows you to monitor training progress in real-time.

%load_ext tensorboard
%tensorboard --logdir {LOCAL_OUTPUT_DIR}

In [ ]:
#@markdown # **Start Training.** 🏋️‍♂️
#@markdown ---
#@markdown This cell runs the Piper training process.

import os
import glob
import re

# Change to piper directory
%cd /content/piper1-gpl

# Build the training command
csv_path = f"{LOCAL_DATASET_DIR}/metadata.csv"
config_path = f"{LOCAL_OUTPUT_DIR}/{model_name}.json"
cache_dir = LOCAL_CACHE_DIR
audio_dir = LOCAL_WAVS_DIR

# Determine checkpoint path
ckpt_path_arg = ""

if resume_training:
    # Look for existing checkpoint
    checkpoints = glob.glob(f"{LOCAL_OUTPUT_DIR}/lightning_logs/**/checkpoints/last.ckpt", recursive=True)
    if checkpoints:
        # Sort by version number to get the latest
        latest_ckpt = sorted(checkpoints, key=lambda x: int(re.findall(r'version_(\d+)', x)[0]) if re.findall(r'version_(\d+)', x) else 0)[-1]
        print(f"Resuming from checkpoint: {latest_ckpt}")
        ckpt_path_arg = f"--ckpt_path \"{latest_ckpt}\""
    else:
        print("No existing checkpoint found. Starting fresh.")
        if use_pretrained and os.path.exists(PRETRAINED_CKPT_PATH):
            ckpt_path_arg = f"--ckpt_path \"{PRETRAINED_CKPT_PATH}\""
elif use_pretrained and os.path.exists(PRETRAINED_CKPT_PATH):
    print(f"Fine-tuning from pretrained checkpoint: {PRETRAINED_CKPT_PATH}")
    ckpt_path_arg = f"--ckpt_path \"{PRETRAINED_CKPT_PATH}\""

# Build the training command using the piper.train module
train_cmd = f"""
python -m piper.train \
    --data.csv_path "{csv_path}" \
    --data.cache_dir "{cache_dir}" \
    --data.audio_dir "{audio_dir}" \
    --data.espeak_voice "{language}" \
    --data.config_path "{config_path}" \
    --data.voice_name "{model_name}" \
    --data.sample_rate {sample_rate} \
    --data.batch_size {batch_size} \
    --data.validation_split {validation_split} \
    --data.num_test_examples {num_test_examples} \
    --model.sample_rate {sample_rate} \
    --model.num_speakers {num_speakers} \
    --trainer.max_epochs {max_epochs} \
    --trainer.accelerator gpu \
    --trainer.devices 1 \
    --trainer.precision 32 \
    --trainer.default_root_dir "{LOCAL_OUTPUT_DIR}" \
    --trainer.callbacks+=ModelCheckpoint \
    --trainer.callbacks.dirpath "{LOCAL_OUTPUT_DIR}/checkpoints" \
    --trainer.callbacks.filename "piper-{{epoch:04d}}-{{step:08d}}" \
    --trainer.callbacks.save_top_k 3 \
    --trainer.callbacks.save_last true \
    --trainer.callbacks.every_n_epochs 1 \
    {ckpt_path_arg}
"""

print("Starting training...")
print(f"Training command:\n{train_cmd}")
!{train_cmd}

# 💾 **5. Save Model to Drive** 💾

In [ ]:
#@markdown # **Export Model to ONNX.** 📦
#@markdown ---
#@markdown This cell exports the trained model to ONNX format for inference.

import os
import glob

# Find the best checkpoint
checkpoint_dir = f"{LOCAL_OUTPUT_DIR}/checkpoints"
last_ckpt = f"{checkpoint_dir}/last.ckpt"

if os.path.exists(last_ckpt):
    best_checkpoint = last_ckpt
    print(f"Using last checkpoint: {best_checkpoint}")
else:
    # Find any checkpoint
    all_checkpoints = glob.glob(f"{LOCAL_OUTPUT_DIR}/**/checkpoints/*.ckpt", recursive=True)
    if all_checkpoints:
        best_checkpoint = sorted(all_checkpoints)[-1]
        print(f"Using checkpoint: {best_checkpoint}")
    else:
        raise Exception(f"No checkpoints found in {LOCAL_OUTPUT_DIR}")

# Export to ONNX
onnx_output_path = f"{LOCAL_OUTPUT_DIR}/{model_name}.onnx"

%cd /content/piper1-gpl
!python -m piper.train.export_onnx \
    --checkpoint "{best_checkpoint}" \
    --output-file "{onnx_output_path}"

print(f"\n✅ Model exported to: {onnx_output_path}")

In [ ]:
#@markdown # **Save Model and Config to Google Drive.** 💾
#@markdown ---
#@markdown This cell copies the trained model and config files to Google Drive.

import os
import shutil

# Create model-specific output directory in Google Drive
gdrive_model_dir = os.path.join(gdrive_output_path, model_name)
os.makedirs(gdrive_model_dir, exist_ok=True)

# Files to copy
files_to_copy = [
    (f"{LOCAL_OUTPUT_DIR}/{model_name}.onnx", f"{gdrive_model_dir}/{model_name}.onnx"),
    (f"{LOCAL_OUTPUT_DIR}/{model_name}.json", f"{gdrive_model_dir}/{model_name}.onnx.json"),
]

# Also copy the last checkpoint
last_ckpt = f"{LOCAL_OUTPUT_DIR}/checkpoints/last.ckpt"
if os.path.exists(last_ckpt):
    files_to_copy.append((last_ckpt, f"{gdrive_model_dir}/last.ckpt"))

# Copy files
for src, dst in files_to_copy:
    if os.path.exists(src):
        print(f"Copying {src} -> {dst}")
        shutil.copy(src, dst)
    else:
        print(f"Warning: {src} not found, skipping.")

print(f"\n✅ Model saved to Google Drive: {gdrive_model_dir}")
print("\nFiles saved:")
for f in os.listdir(gdrive_model_dir):
    fpath = os.path.join(gdrive_model_dir, f)
    size_mb = os.path.getsize(fpath) / (1024 * 1024)
    print(f"  - {f} ({size_mb:.2f} MB)")

# 🎧 **6. Test the Model (Optional)** 🎧

In [ ]:
#@markdown # **Test the Exported Model.** 🎧
#@markdown ---
#@markdown This cell tests the exported ONNX model by generating speech.

#@markdown ### Text to synthesize:
test_text = "Hello, this is a test of the fine-tuned Piper text to speech model." #@param {type:"string"}

import json
import numpy as np
from IPython.display import Audio, display

try:
    import onnxruntime as ort
except ImportError:
    !pip install -q onnxruntime
    import onnxruntime as ort

# Load model and config
onnx_path = f"{LOCAL_OUTPUT_DIR}/{model_name}.onnx"
config_json_path = f"{LOCAL_OUTPUT_DIR}/{model_name}.json"

if not os.path.exists(onnx_path):
    raise Exception(f"ONNX model not found: {onnx_path}")

if not os.path.exists(config_json_path):
    raise Exception(f"Config file not found: {config_json_path}")

# Load config
with open(config_json_path, 'r') as f:
    config = json.load(f)

phoneme_id_map = config.get('phoneme_id_map', {})
model_sample_rate = config.get('audio', {}).get('sample_rate', 22050)

# Create ONNX session
session = ort.InferenceSession(onnx_path)

# Simple phoneme-based synthesis (for testing)
# In production, use espeak-ng for proper phonemization
def text_to_phoneme_ids(text, phoneme_id_map):
    """Simple text to phoneme ID conversion for testing."""
    ids = []
    for char in text.lower():
        if char in phoneme_id_map:
            ids.extend(phoneme_id_map[char])
        elif char == ' ':
            if ' ' in phoneme_id_map:
                ids.extend(phoneme_id_map[' '])
    return ids if ids else [1, 2, 3, 4, 5]  # Fallback

# Get phoneme IDs
phoneme_ids = text_to_phoneme_ids(test_text, phoneme_id_map)

# Prepare inputs
input_array = np.array([phoneme_ids], dtype=np.int64)
input_lengths = np.array([len(phoneme_ids)], dtype=np.int64)
scales = np.array([0.667, 1.0, 0.8], dtype=np.float32)  # noise_scale, length_scale, noise_scale_w

# Run inference
inputs = {
    'input': input_array,
    'input_lengths': input_lengths,
    'scales': scales,
}

output = session.run(None, inputs)
audio = output[0].squeeze()

print(f"Generated audio shape: {audio.shape}")
print(f"Audio duration: {len(audio) / model_sample_rate:.2f} seconds")

# Play audio
display(Audio(audio, rate=model_sample_rate))

# 📋 **Summary**

This notebook provides a complete pipeline for fine-tuning Piper TTS with data from Google Drive:

1. **Anti-Disconnect** - Keep Colab session alive
2. **GPU Check** - Verify GPU availability
3. **Mount Drive** - Access your Google Drive
4. **Install Software** - Install Piper and dependencies from the piper1-gpl repository
5. **Dataset Loading** - Copy dataset from Drive to local storage
6. **Training** - Fine-tune the model with configurable parameters
7. **Export & Save** - Export to ONNX and save to Google Drive

### Output Files

After training, you'll find these files in your Google Drive output folder:
- `{model_name}.onnx` - The trained model in ONNX format
- `{model_name}.onnx.json` - Model configuration file
- `last.ckpt` - Latest training checkpoint (for resuming training)

### Using the Model

To use the trained model with Piper:
```bash
echo "Hello world" | piper --model {model_name}.onnx --output_file output.wav
```